# 正解データ分析

    

# 1. ライブラリ導入

In [19]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import geopandas as gpd
from shapely.geometry import box
import contextily as ctx

import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams




FONT_PATH = "/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"
font_prop = font_manager.FontProperties(fname=FONT_PATH)

rcParams["font.family"] = font_prop.get_name()


## 1. 調布市実際の滞在人数データをロード

In [7]:
DATE_YYYYMMDD = "20190208"   # ←対象の日付に変更

ROOT_DIR = Path("/Users/osamu/study/poi_sim/data/processed/yamada_processed/trans_count")

# 自動でファイル名を作る
STATE_COUNT_PATH = ROOT_DIR / f"state_count_{DATE_YYYYMMDD}.csv"

print("✅ target file:", STATE_COUNT_PATH)


df = pd.read_csv(STATE_COUNT_PATH)

df["time_15min"] = pd.to_datetime(df["time_15min"], errors="coerce")
df = df.dropna(subset=["time_15min"]).copy()

df["time_hm"] = df["time_15min"].dt.strftime("%H:%M")
df["state_mesh"] = df["state_mesh"].astype(str)

print("✅ loaded rows:", len(df))
display(df.head())




✅ target file: /Users/osamu/study/poi_sim/data/processed/yamada_processed/trans_count/state_count_20190208.csv
✅ loaded rows: 29630


,time_15min,state_mesh,state_poi,next_mesh,next_poi,n_transition,time_hm
0,2019-02-08,11111111,OoR,11111111.0,OoR,42,00:00
1,2019-02-08,11111111,OoR,53393472.0,move,1,00:00
2,2019-02-08,11111111,OoR,53393474.0,home,1,00:00
3,2019-02-08,11111111,OoR,53393474.0,move,1,00:00
4,2019-02-08,11111111,OoR,53393475.0,move,1,00:00


In [9]:
gt_mesh_time_proxy = (
    df.groupby(["time_hm", "state_mesh"], as_index=False)["n_transition"]
    .sum()
    .rename(columns={"state_mesh": "mesh", "n_transition": "n_people"})
)

print("✅ proxy mesh-time shape:", gt_mesh_time_proxy.shape)
display(gt_mesh_time_proxy.head())
print("time steps:", gt_mesh_time_proxy["time_hm"].nunique())

gt_mesh_time_proxy = gt_mesh_time_proxy[gt_mesh_time_proxy["mesh"] != "11111111"].copy()
print("✅ after remove 11111111:", gt_mesh_time_proxy.shape)



✅ proxy mesh-time shape: (3270, 3)


,time_hm,mesh,n_people
0,00:00,11111111,457
1,00:00,53393463,4
2,00:00,53393464,15
3,00:00,53393471,5
4,00:00,53393472,47


time steps: 95
✅ after remove 11111111: (3175, 3)


In [13]:

# -------------------------
# 3次メッシュ(8桁) → bbox
# -------------------------
def mesh3_to_bbox(mesh3: str):
    mesh3 = str(mesh3)
    if len(mesh3) != 8:
        return None

    p = int(mesh3[0:2])
    q = int(mesh3[2:4])

    lat0 = p / 1.5
    lon0 = q + 100

    r = int(mesh3[4])
    s = int(mesh3[5])
    lat0 += r * (5/60)
    lon0 += s * (7.5/60)

    t = int(mesh3[6])
    u = int(mesh3[7])
    lat0 += t * (30/3600)
    lon0 += u * (45/3600)

    lat1 = lat0 + (30/3600)
    lon1 = lon0 + (45/3600)

    return (lon0, lat0, lon1, lat1)

# -------------------------
# gdf_mesh 作成
# -------------------------
unique_mesh = gt_mesh_time_proxy["mesh"].unique()

rows = []
ng = 0
for m in unique_mesh:
    if m == "11111111":   # ✅ 異常メッシュは除外
        continue
    bb = mesh3_to_bbox(m)
    if bb is None:
        ng += 1
        continue
    rows.append({"mesh": m, "geometry": box(*bb)})

gdf_mesh = gpd.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")

print("✅ gdf_mesh created:", gdf_mesh.shape, "NG mesh:", ng)
display(gdf_mesh.head())


✅ gdf_mesh created: (39, 2) NG mesh: 0


,mesh,geometry
0,53393463,"POLYGON ((139.55 35.63333, 139.55 35.64167, 13..."
1,53393464,"POLYGON ((139.5625 35.63333, 139.5625 35.64167..."
2,53393471,"POLYGON ((139.525 35.64167, 139.525 35.65, 139..."
3,53393472,"POLYGON ((139.5375 35.64167, 139.5375 35.65, 1..."
4,53393473,"POLYGON ((139.55 35.64167, 139.55 35.65, 139.5..."


In [20]:
# タイトル用（日付ラベル）
DATE_LABEL = f"{DATE_YYYYMMDD[:4]}-{DATE_YYYYMMDD[4:6]}-{DATE_YYYYMMDD[6:8]}"

time_list = sorted(gt_mesh_time_proxy["time_hm"].unique())
global_vmax = gt_mesh_time_proxy["n_people"].max()

@interact(
    t=widgets.SelectionSlider(options=time_list, value=time_list[0], description="時刻")
)
def plot_time_bg(t):
    df_t = gt_mesh_time_proxy[gt_mesh_time_proxy["time_hm"] == t].copy()

    # join
    gdf_t = gdf_mesh.merge(df_t, on="mesh", how="left")
    gdf_t["n_people"] = gdf_t["n_people"].fillna(0)

    # ✅ 背景地図用に投影変換
    gdf_web = gdf_t.to_crs(epsg=3857)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))

    # ✅ 背景地図を先に描く
    # extent を gdf の範囲に合わせる
    ax.set_xlim(gdf_web.total_bounds[0], gdf_web.total_bounds[2])
    ax.set_ylim(gdf_web.total_bounds[1], gdf_web.total_bounds[3])
    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    # ✅ メッシュを上に重ねる（線あり）
    gdf_web.plot(
        column="n_people",
        ax=ax,
        legend=True,
        vmin=0,
        vmax=global_vmax,
        linewidth=0.8,
        edgecolor="black",
        alpha=0.55
    )

    ax.set_title(f"{DATE_LABEL} 遷移元カウント（proxy）@ {t}", fontsize=14)
    ax.set_axis_off()

    plt.tight_layout()
    plt.show()


interactive(children=(SelectionSlider(description='時刻', options=('00:00', '00:15', '00:30', '00:45', '01:00', …